Below I implement a random reflex agent. The domain is passed via a text file called floorplan.txt. There are a few parameters to be mindful of: The chance of the 'Suck' operation executing and the chance of a clean square becoming dirty again. I was carfull to put this functionality in the "correct" place. For instance it is the environments responsibility to get dirty again, but the agents responsibility to have a faulty suck operation that only works 80% of the time. 

In [21]:
import numpy as np
from agents import *

class VacuumAgent(Agent):

    def __init__(self, program = None):
        super().__init__(program)
        self.location = [1,1]
        self.direction = 'R' # 'R', 'L', 'U', 'D'

def program(percept):
    if percept == 2:
        chance = random.randint(1,100)
        if(chance > 20):
            return ('Suck','None')
        else:
            return ('None','None')
    else:
        bump = 'Bump' if agent.bump else 'None'
        return ('Move', bump)

class Environment:
    
    def __init__(self):
        self.agents = []
        
        with open('floorplan.txt') as f:
            floor_plan = np.array([list(map(int, line.strip())) for line in f])
        self.floor_plan = floor_plan
        
    def percept(self, agent):
        return self.floor_plan[agent.location[0]][agent.location[1]]

    def execute_action(self, agent, action):
        if action[0] == 'Suck':
            self.floor_plan[agent.location[0]][agent.location[1]] = 1
            agent.performance += 100
        
        elif action[0] == 'Move':
            agent.performance -= 1
            if action[1] == 'Bump':
                agent.bump = False
            else:
                agent.direction = random.choice(['R', 'L', 'U', 'D'])
                agent.bump = self.move_agent(agent)
                
    def add_dirt(self):
        x_len = len(self.floor_plan)
        y_len = len(self.floor_plan[0])
        for i in range(x_len):
            for j in range(y_len):
                if(self.floor_plan[i][j] == 1):
                    chance = random.randint(1, 100)
                    if(chance < 5):
                        self.floor_plan[i][j] = 2
                    
    
    def move_agent(self, agent):
        if agent.direction == 'R':
            loc = agent.location
            if self.floor_plan[loc[0]][loc[1] + 1] == 0:
                agent.bump = True
            else:
                agent.location[1] += 1
        elif agent.direction == 'L':
            loc = agent.location
            if self.floor_plan[loc[0]][loc[1] - 1] == 0:
                agent.bump = True
            else:
                agent.location[1] -= 1
        elif agent.direction == 'U':
            loc = agent.location
            if self.floor_plan[loc[0] - 1][loc[1]] == 0:
                agent.bump = True
            else:
                agent.location[0] -= 1
        elif agent.direction == 'D':
            loc = agent.location
            if self.floor_plan[loc[0] + 1][loc[1]] == 0:
                agent.bump = True
            else:
                agent.location[0] += 1
        return agent.bump

    def step(self):
        action = agent.program(self.percept(agent))
        self.add_dirt()
        self.execute_action(self.agents[0], action)

    def run(self,steps=100):
        for step in range(steps):
            self.step()
    
    def display_floor(self):
        print(self.floor_plan)

    def add_agent(self, agent):
        self.agents.append(agent)


env = Environment()
agent = VacuumAgent(program)
env.add_agent(agent)
print("floor before:")
env.display_floor()
env.run()

print("floor after:")
env.display_floor()
print("performance score: ",agent.performance)

floor before:
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 1 2 1 1 1 1 1 1 1 1 1 1 0]
 [0 0 1 0 1 0 1 0 2 2 2 2 2 2 0]
 [0 0 1 0 1 1 2 1 1 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
floor after:
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 2 2 2 2 2 1 1 2 2 2 2 2 2 0]
 [0 0 2 0 1 0 1 0 2 2 2 2 2 2 0]
 [0 0 2 0 2 2 1 2 2 2 2 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
performance score:  1519
